In [4]:
import numpy as np
import cv2
import glob
import re
from os import path as ospath
from utils import get_glass_height_cm
from utils import get_glass_height_mm
from utils import center_crop

The `calculate_liquid_height_simple` function processes an image of a glass containing liquid and estimates the **liquid height** in **millimeters** based on the known height of the glass.

---

### Steps Performed:
1. **Image Preprocessing:**
   - Loads the input image.
   - Converts it to grayscale.
   - Applies Gaussian Blur to reduce noise.
   - Saves the preprocessed grayscale image for debugging.

2. **Glass Detection:**
   - Uses **Sobel edge detection** to extract glass edges.
   - Applies **contour detection** to find the outer shape of the glass.
   - Merges contours and applies **convex hull** to ensure a complete boundary.
   - Draws and saves the **hull contour** for debugging.
   - Computes a **bounding box** around the detected glass.

3. **Liquid Segmentation:**
   - Extracts the **glass region of interest (ROI)** from the grayscale image.
   - Uses **Otsu’s thresholding** to detect the **liquid region**.
   - Saves the **liquid mask** for debugging.

4. **Liquid Height Calculation:**
   - Finds the largest contour inside the **liquid mask**.
   - Determines the **bounding box** around the liquid region.
   - Computes the **height of the liquid in pixels**.
   - Converts pixel height to **millimeters** using the **glass height** as a reference.

5. **Debugging and Visualization:**
   - Draws **bounding rectangles** for both the **glass** and **liquid**.
   - Saves the **debug image** with highlighted regions.

6. **Returns:**  
   - The **liquid height in millimeters**.


In [5]:
def calculate_liquid_height_simple(image_path, debug_name = 'out.jpg', glass_height_mm = 88.0):
    # Load the image
    image = cv2.imread(image_path)

    # Convert to grayscale and apply Gaussian Blur to reduce noise
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)

    cv2.imwrite(f'out/simple/gray-{debug_name}', gray)
    
    # Detect edges using Sobel
    grad_x = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
    grad_y = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)
    gradient = cv2.magnitude(grad_x, grad_y)
    _, glass_edges = cv2.threshold(np.uint8(gradient), 30, 255, cv2.THRESH_BINARY)

    cv2.imwrite(f'out/simple/glass-{debug_name}', glass_edges)
    
    # Find contours in the glass edge mask
    contours, _ = cv2.findContours(glass_edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    merged_contour = []
    for contour in contours:
        if cv2.contourArea(contour) > 500:
            merged_contour.extend(contour)

    merged_contour = np.array(merged_contour)
    hull = cv2.convexHull(merged_contour)

    cv2.imwrite(f'out/simple/hull-{debug_name}', cv2.drawContours(np.zeros_like(image), [hull], -1, (255), thickness=2))
    
    if len(contours) == 0:
        raise ValueError("Glass NOT found")
    
    
    # Bounding box around the detected glass
    x, y, w, h = cv2.boundingRect(hull)
    glass_height_px = h

    # Extract region of interest (ROI) containing the glass
    glass_roi = gray[y:y+h, x:x+w]

    # Segment the liquid using thresholding
    _, liquid_mask = cv2.threshold(glass_roi, 50, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    cv2.imwrite(f'out/simple/liquid-mask-{debug_name}', liquid_mask)

    # Get the largest contour
    liquid_contours, _ = cv2.findContours(liquid_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    liquid_contour = max(liquid_contours, key=cv2.contourArea)
    
    # Find the bounding rectangle for the liquid
    _, liquid_y, _, liquid_h = cv2.boundingRect(liquid_contour)
    liquid_height_px = liquid_h  # Liquid height in pixels

    # Convert from pixels to millimeters
    px_per_mm = glass_height_px / glass_height_mm
    liquid_height_mm = liquid_height_px / px_per_mm

    # Draw debug rectangles
    debug_image = image.copy()
    cv2.rectangle(debug_image, (x, y), (x + w, y + h), (0, 255, 0), 2)  # Glass
    cv2.rectangle(debug_image, (x, y + liquid_y), (x + w, y + liquid_y + liquid_h), (255, 0, 0), 2)
    cv2.imwrite(f'out/simple/debug-{debug_name}', debug_image)

    return liquid_height_mm

### Problems:

While this approach provides a straightforward method for estimating liquid height, it introduces a significant margin of error compared to true measurements. Several factors contribute to these inaccuracies:

1. **Camera Angle Deviation**  
   Despite efforts to capture images at a **90-degree angle**, slight misalignments were unavoidable. As a result, the camera was not always perfectly perpendicular to the glass, leading to distortions in both the glass and the liquid, ultimately affecting measurement precision.

2. **Presence of Noise**  
   The images contain **visual noise**, including background elements such as the table, shadows, and reflections. These artifacts interfere with the segmentation process, making it challenging to accurately detect the glass and liquid boundaries.

3. **Lack of Camera Calibration**  
   No **intrinsic** or **extrinsic** camera calibration was applied. Without correcting for these parameters, images remain affected by **radial and tangential lens distortions**, which can distort object dimensions and introduce further inaccuracies.

### Main script
Here we determine the liquid height in a glass using the images collected in our dataset. We begin by retrieving all image files, where the filename structure follows X-Y.jpg. The first character in the filename represents the type of glass (A, B, or C), which is extracted using a regular expression. Based on this identifier, the known height of the corresponding glass is retrieved in millimeters. The script then processes each image by applying the calculate_liquid_height_simple function, which determines the liquid height using pixel-to-metric conversions.

In [6]:
filelist = glob.glob("images/?-?.jpg")

for file in filelist:
    type_img = re.search('([A-C]?)-[A-C]?', file).group(1)
    height = get_glass_height_mm(type_img)
    
    liquid_height_mm = calculate_liquid_height_simple(file, debug_name = ospath.basename(file), glass_height_mm = height)
    print(f"File: {file}, Glass Type: {type_img}, Liquid Height: {liquid_height_mm:.2f} mm")

File: images\A-A.jpg, Glass Type: A, Liquid Height: 45.41 mm
File: images\A-C.jpg, Glass Type: A, Liquid Height: 35.17 mm
File: images\A-F.jpg, Glass Type: A, Liquid Height: 22.51 mm
File: images\B-A.jpg, Glass Type: B, Liquid Height: 43.40 mm
File: images\B-C.jpg, Glass Type: B, Liquid Height: 39.17 mm
File: images\B-F.jpg, Glass Type: B, Liquid Height: 22.81 mm
File: images\C-A.jpg, Glass Type: C, Liquid Height: 84.85 mm
File: images\C-C.jpg, Glass Type: C, Liquid Height: 43.19 mm
File: images\C-F.jpg, Glass Type: C, Liquid Height: 30.30 mm


We perform camera calibration using multiple images of a checkerboard pattern. The process begins by defining the checkerboard's real-world coordinates and setting up termination criteria for corner refinement. It then iterates through a set of calibration images, detecting and refining the checkerboard corners using OpenCV’s `findChessboardCorners` and `cornerSubPix` functions. The detected corners are stored and later used in OpenCV’s `calibrateCamera` function, which estimates the camera's intrinsic and extrinsic parameters. The calibration results include the camera matrix, distortion coefficients, rotation, and translation vectors, which can be used for undistorting images and improving measurement accuracy.
Below, we provide a detailed description of the main functions used in the calibration process.

Description of the parameters for `findChessboardCorners`

| **Parameter**   | **Description**  |
|-----------------|------------------|
| `image`         | The input image containing the checkerboard pattern. It must be an 8-bit grayscale or color image. |
| `patternSize`   | The number of inner corners per checkerboard row and column, defined as: (`patternSize = cvSize(columns, rows)`). |
| `corners`       | The output array containing the detected corner coordinates. |
| `flags`         | Various operation flags. Typically, the default settings work unless additional tuning is needed. |

**Returned Values of `findChessboardCorners`**
- **`retval`**: A boolean value (true or false) indicating whether the checkerboard pattern was
successfully detected.
- **`corners`**: An array of 2D points representing the detected checkerboard corners in the image.

---

Description of the parameters for `cornerSubPix`

| **Parameter**  | **Description**  |
|---------------|----------------|
| `image`       | The input image in which corners are refined. |
| `corners`     | Initial corner coordinates; the function updates these with the refined positions. |
| `winSize`     | Half the size of the search window used for refining the corner position. |
| `zeroZone`    | Defines a dead region in the middle of the search zone where the summation is not performed to avoid singularities. A value of (-1, -1) means no such region. |
| `criteria`    | The termination criteria for the iterative search, stopping when the desired accuracy is reached or a maximum number of iterations is exceeded. |

---

Description of the parameters for `calibrateCamera`

| **Parameter**     | **Description**  |
|------------------|----------------|
| `objectPoints`   | A vector of vectors containing the 3D coordinates of the checkerboard corners in world space. |
| `imagePoints`    | A vector of vectors containing the 2D pixel coordinates of the checkerboard corners in the captured images. |
| `imageSize`      | The resolution of the image in pixels. |
| `cameraMatrix`   | The intrinsic camera matrix, which contains parameters such as focal length and optical center. |
| `distCoeffs`     | The distortion coefficients, which correct for lens distortions. |
| `rvecs`          | A set of rotation vectors (3 × 1) that define the camera’s orientation for each image. |
| `tvecs`          | A set of translation vectors (3 × 1) that define the camera’s position relative to the checkerboard. |


**Returned Values of `calibrateCamera`**
- **`retval`**: The reprojection error, which measures the accuracy of the calibration.
- **`cameraMatrix`**: The intrinsic matrix that describes the internal properties of the camera.
- **`distCoeffs`**: The distortion coefficients for correcting radial and tangential distortions.
- **`rvecs` and `tvecs`**: The rotation and translation vectors, which define the pose of the camera relative to the checkerboard in each captured image.


In [7]:
# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
 
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((7*7,3), np.float32)
objp[:,:2] = np.mgrid[0:7,0:7].T.reshape(-1,2)
 
# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.
 
images = glob.glob('images/chess-*.jpg')
 
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
 
    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, (7,7), None)
 
    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)
 
        corners2 = cv2.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners2)
 
        # Draw and display the corners
        cv2.drawChessboardCorners(img, (7,7), corners2, ret)
        cv2.imwrite(f'out/corners-{ospath.basename(fname)}', img)
        
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

The `calculate_liquid_height_complex` function is a more refined approach for estimating **liquid height in centimeters** from an image of a glass. This function improves upon the simpler previous method by incorporating **camera calibration** preprocessing techniques, such as noise reduction and camera calibration as to achieve more precise measurements.

---

### Process:

**1. Camera Calibration & Image Preprocessing**
- Loads the input image.
- **Undistorts the image** using camera matrix (`mtx`) and distortion coefficients (`dist`).
- Crops the **Region of Interest (ROI)** to remove black borders after correction.
- Resizes and centers the image to focus on the relevant area.
- Converts the image to **grayscale** and applies **Gaussian blur** to reduce noise.

**2. Edge Detection & Glass Contour Extraction**
- Uses **Sobel filtering** to detect **sharp edges** in both x and y directions.
- Applies **morphological operations** (closing and opening) to clean the edges.
- Detects contours and applies **convex hull** to obtain the most complete boundary of the glass.
- Saves the **detected glass hull** for debugging.

**3. Identifying the Glass & Liquid Region**
- Finds the **bounding rectangle** around the detected glass.
- Extracts the **glass region** from the grayscale image.
- Applies **Otsu’s thresholding** to separate the **liquid region** from the glass.
- Uses **morphological operations** to remove noise, small shadows, and refine the liquid segmentation.

**4. Detecting the Liquid Surface**
- Finds **top and bottom boundaries** of the liquid.
- Excludes **shadows or artifacts** from the detected liquid mask.
- Combines the **liquid mask with the glass mask** for better refinement.

**5. Liquid Height Calculation**
- Identifies the **largest liquid contour**.
- Computes the **bounding box** around the liquid region.
- Converts the **pixel height to centimeters** using the calibrated **pixels per cm** ratio.

**6. Debugging & Visualization**
- Saves **intermediate outputs** (e.g., **glass edges, hull, liquid mask**).
- Draws **bounding boxes** around both the **glass** (green) and **liquid** (blue).
- Saves the **final debug image** for visualization.

**7. Returns:**
- The estimated **liquid height in centimeters**.

---

## Improvements
**Uses camera calibration** to correct for lens distortions.  
**Morphological operations** in order to improve edge detection and segmentation.  
**Refines glass and liquid surface detection** by removing background noise and shadows.  

This method provides a **more reliable and accurate** liquid height estimation as can be seen by the debug images


In [ ]:
def calculate_liquid_height_complex(image_path, debug_name = 'out.jpg', glass_height_mm = 10):

    img = cv2.imread(image_path)
    h, w = img.shape[:2]
    newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))

    # Undistort the image
    image = cv2.undistort(img, mtx, dist, None, newcameramtx)

    # Crop the image
    x, y, w, h = roi
    image = image[y:y+h, x:x+w]
    
    cv2.imwrite(f'out/camera_cal/calib-{ospath.basename(file)}', image)
    
    # Image resize
    scale_percent = 50 
    width = int(image.shape[1] * scale_percent / 100)
    height = int(image.shape[0] * scale_percent / 100)
    image = cv2.resize(image, (width, height))

    image = center_crop(image, width - 500, height - 100)

    # Grayscale conversion and Gaussian blur
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (7, 7), 0)

    # Edge detection
    grad_x = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
    grad_y = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)
    gradient = cv2.magnitude(grad_x, grad_y)

    threshold = np.percentile(gradient, 98.75)
    _, glass_edges = cv2.threshold(np.uint8(gradient), threshold, 255, cv2.THRESH_BINARY)

    # Morphological operations
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
    glass_edges = cv2.morphologyEx(glass_edges, cv2.MORPH_CLOSE, kernel)
    glass_edges = cv2.morphologyEx(glass_edges, cv2.MORPH_OPEN, kernel)
    glass_edges = cv2.dilate(glass_edges, kernel, iterations=1)

    cv2.imwrite(f'out/camera_cal/glass-{debug_name}', glass_edges)

    # Glass contours
    contours, _ = cv2.findContours(glass_edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    merged_contour = []
    for contour in contours:
        if cv2.contourArea(contour) > 500:
            merged_contour.extend(contour) 

    merged_contour = np.array(merged_contour)
    hull = cv2.convexHull(merged_contour)
    cv2.imwrite(f'out/camera_cal/hull-{debug_name}', cv2.drawContours(np.zeros_like(image), [hull], -1, (255), thickness=2))
    if len(contours) == 0:
        raise ValueError("No glass detected.")

    # Bounding rectangle for the glass
    x, y, w, h = cv2.boundingRect(hull)
    glass_height_px = h

    # ROI of the glass
    roi_gray = gray[y:y+h, x:x+w]

    # Threshold to find the liquid
    _, liquid_mask = cv2.threshold(roi_gray, 50, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    
    # Morphological operations
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    liquid_mask = cv2.erode(liquid_mask, kernel, iterations=5)
    liquid_mask = cv2.morphologyEx(liquid_mask, cv2.MORPH_CLOSE, kernel)
    liquid_mask = cv2.morphologyEx(liquid_mask, cv2.MORPH_OPEN, kernel)
    cv2.imwrite(f'out/camera_cal/liquid-mask-{debug_name}', liquid_mask)

    # Liquid boundaries
    liquid_y, liquid_x = np.where(liquid_mask > 0)
    if len(liquid_y) == 0 or len(liquid_x) == 0:
        raise ValueError("Liquid mask is empty.")
    liquid_top = np.min(liquid_y)
    liquid_bottom = np.max(liquid_y)

    # Ignore shadows mask
    shadow_threshold = int((liquid_bottom - liquid_top) * 0.08)
    refined_mask = np.zeros_like(liquid_mask)
    refined_mask[liquid_top:liquid_bottom - shadow_threshold, :] = liquid_mask[liquid_top:liquid_bottom - shadow_threshold, :]

    liquid_only_mask = cv2.bitwise_and(refined_mask, liquid_mask)

    cv2.imwrite(f'out/camera_cal/liquid-{debug_name}', liquid_only_mask)

    # Liquid contour
    liquid_contours, _ = cv2.findContours(liquid_only_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if len(liquid_contours) == 0:
        raise ValueError("No liquid detected.")
    liquid_contour = max(liquid_contours, key=cv2.contourArea)

    # Bounding rectangle for the liquid
    _, liquid_y, _, liquid_h = cv2.boundingRect(liquid_contour)
    liquid_height_px = liquid_h 

    # PPM
    px_per_mm = glass_height_px / glass_height_mm
    liquid_height_mm = liquid_height_px / px_per_mm

    # Debug visualization
    debug_image = image.copy()
    cv2.rectangle(debug_image, (x, y), (x + w, y + h), (0, 255, 0), 2)
    cv2.rectangle(debug_image, (x, y + liquid_y), (x + w, y + liquid_y + liquid_h), (255, 0, 0), 2)
    cv2.imwrite(f'out/camera_cal/debug-{debug_name}', debug_image)

    return liquid_height_mm

In [ ]:
filelist = glob.glob("images/?-?.jpg")

for file in filelist:
    type_img = re.search('([A-C]?)-[A-C]?', file).group(1)
    height = get_glass_height_mm(type_img)
    
    liquid_height_mm = calculate_liquid_height_complex(file, debug_name = ospath.basename(file), glass_height_mm = height)
    print(f"File: {file}, Glass Type: {type_img}, Liquid Height: {liquid_height_mm:.2f} mm")

File: images\A-A.jpg, Glass Type: A, Liquid Height: 31.38 mm
File: images\A-C.jpg, Glass Type: A, Liquid Height: 45.93 mm
File: images\A-F.jpg, Glass Type: A, Liquid Height: 19.80 mm
File: images\B-A.jpg, Glass Type: B, Liquid Height: 34.43 mm
File: images\B-C.jpg, Glass Type: B, Liquid Height: 45.48 mm
File: images\B-F.jpg, Glass Type: B, Liquid Height: 21.70 mm
File: images\C-A.jpg, Glass Type: C, Liquid Height: 37.36 mm
File: images\C-C.jpg, Glass Type: C, Liquid Height: 50.41 mm
File: images\C-F.jpg, Glass Type: C, Liquid Height: 25.68 mm
